In [1]:
import pickle
import sys,json
import pywt
from keras.models import load_model
import numpy as np
import librosa
emotion_model_path = 'emot_model.h5'
emotion_classifier = load_model(emotion_model_path, compile=False)
df = pickle.load(open("recommend",'rb'))
wi = pickle.load(open("token1",'rb'))
df1 = pickle.load(open('recom','rb'))
t_em_path = 'text_emotion1.h5'
tem_classifier = load_model(t_em_path, compile=False)
import pandas as pd

In [2]:
def zcr(data, frame_length=2048, hop_length=512):
    zcr = librosa.feature.zero_crossing_rate(y=data, frame_length=frame_length, hop_length=hop_length)
    return np.squeeze(zcr)
#RMS Energy
def rmse(data, frame_length=2048, hop_length=512):
    rmse = librosa.feature.rms(y=data, frame_length=frame_length, hop_length=hop_length)
    return np.squeeze(rmse)
#MFCC
def mfcc(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    mfcc_feature = librosa.feature.mfcc(y=data, sr=sr)
    return np.squeeze(mfcc_feature.T) if not flatten else np.ravel(mfcc_feature.T)

In [3]:
def extract_features(y, sr, frame_length=2048, hop_length=512):
    result = np.array([])
    result = np.hstack((result,
                        zcr(y, frame_length, hop_length),
                        rmse(y, frame_length, hop_length),
                        mfcc(y, sr, frame_length, hop_length)
                                    ))
    return result

In [4]:
import pyaudio
import numpy as np
import librosa
import wave
from tensorflow.image import resize
from scipy.ndimage import zoom
def get_audio():
    Chunk = 1024
    Format = pyaudio.paInt16
    Channels = 1
    Rate = 44100
    p = pyaudio.PyAudio()
    stream = p.open(format=Format, channels=Channels, rate=Rate, input=True, frames_per_buffer=Chunk)
    frames = []
    seconds = 3

    print("Recording started")
    for i in range(0, int(Rate / Chunk * seconds)):
        data = stream.read(Chunk)
        frames.append(data)

    print("Recording stopped")
    stream.stop_stream()
    stream.close()
    p.terminate()
    wf = wave.open("o.wav",'wb')
    wf.setnchannels(Channels)
    wf.setsampwidth(p.get_sample_size(Format))
    wf.setframerate(Rate)
    wf.writeframes(b"".join(frames))
    wf.close()
    audio_data = b"".join(frames)  # Combine the recorded frames into a single byte string

    # Convert the byte string to a NumPy array of int16
    audio_array = np.frombuffer(audio_data, dtype=np.int16)

    # Convert the int16 array to floating-point values between -1 and 1
    audio_array = audio_array.astype(np.float32) / 32768.0
    # Compute MFCC
    mfcc = extract_features(audio_array,sr=Rate)

    # Add an extra dimension to match the expected input shape of the "DCNN" layer
    mfcc0 = np.resize(mfcc,(2376,1))
    mfcc_reshaped = np.reshape(mfcc0, (1, 2376, 1))
    x=emotion_classifier(mfcc_reshaped)
    emotion_code=np.argmax(x,axis=1)
    NUM_RECOMMEND=10
    sset=[]
    if emotion_code==2:#neutral
        sset.append(df[df['kmeans']==0]['name'].head(NUM_RECOMMEND))
        sset=pd.DataFrame(sset).T
    elif emotion_code==1:#happy
        sset.append(df[df['kmeans']==1]['name'].head(NUM_RECOMMEND))
        sset=pd.DataFrame(sset).T
    elif emotion_code==3:#sad
        sset.append(df[df['kmeans']==2]['name'].head(NUM_RECOMMEND))
        sset=pd.DataFrame(sset).T
    elif emotion_code==4:#surprise
        sset.append(df[df['kmeans']==3]['name'].head(NUM_RECOMMEND))
        sset=pd.DataFrame(sset).T
    else:#angry
        sset.append(df[df['kmeans']==4]['name'].head(NUM_RECOMMEND))
        sset= pd.DataFrame(sset).T
    return sset

In [84]:
get_audio()

Recording started
Recording stopped


,name
87747,lovely (with Khalid)
87870,when the party's over
87653,Clean White Noise - Loopable with no fade
88039,Lose You To Love Me
87772,Get You The Moon (feat. Snøw)
97788,No Time To Die
97549,Can We Kiss Forever?
87829,You Are The Reason
87906,i love you
97445,Dusk Till Dawn - Radio Edit


In [82]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

tokenizer = Tokenizer(num_words=8150)
tokenizer.word_index = wi

print(tokenizer.word_index['hello'])

def get_txt():
    p = input("enter text: ")
    sequences = tokenizer.texts_to_sequences([p])
    tweets = pad_sequences(sequences,maxlen=52)  
    x = tem_classifier(tweets)
    emotion_code=np.argmax(x,axis=1)
    NUM_RECOMMEND=10
    hset=[]
    if emotion_code==0:
        #sad
        hset.append(df1[df1['kmeans']==2]['name'].head(NUM_RECOMMEND))
        hset = pd.DataFrame(hset).T
    elif emotion_code==1:#neutral
        hset.append(df1[df1['kmeans']==0]['name'].head(NUM_RECOMMEND))
        hset = pd.DataFrame(hset).T
    else:#happy
        hset.append(df1[df1['kmeans']==1]['name'].head(NUM_RECOMMEND))
        hset= pd.DataFrame(hset).T
    return hset

1897


In [77]:
get_txt()

enter text: had a bad day


,name
87949,death bed (coffee for your head) (feat. beabad...
87856,Someone You Loved
87857,everything i wanted
88000,Skechers
87747,lovely (with Khalid)
87955,Chicago Freestyle (feat. Giveon)
87752,rockstar (feat. 21 Savage)
87870,when the party's over
87891,7 rings
87653,Clean White Noise - Loopable with no fade
